# molecular descriptors for ALDH1 inhibitors:

Different descriptors can be determined on the given molecules to find a (causal) relation between the molecule and the ability to inhibit ALDH1. 

Molecular descriptors are for instance: 
-   molecular mass
-   nr carbon atoms
-   nr hydrogen atoms  
-   nr of bonds
-   nr of branches
-   nr double bindings
-   nr triple bindings
-   cyclic structures
-   Aromaticity (indicated by lower letters)
    -   aromatic nitrogen
-   (tetra hedral) chirality
- nr of rings (e.g. cubane)

### rdkit has automatic implemented descriptors and Fingerprints:

This is used now for the generation of descriptors. also a couple of fingerprint variables can be included. 


In [1]:
import numpy as np
import pandas as pd
from tkinter import filedialog as fd
from rdkit import Chem, DataStructs, RDLogger
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw, Descriptors, AllChem
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator

IPythonConsole.ipython_useSVG=True



In [2]:
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score, matthews_corrcoef
import joblib

In [1]:
import MachineLearning as ml



In [2]:
top100Mols = ml.top100molecules('RandomForestClassifier_PCA1_std.pkl')


2023-06-18 22:47:12.593 python[26235:1164574] +[CATransaction synchronize] called within transaction
2023-06-18 22:47:12.816 python[26235:1164574] +[CATransaction synchronize] called within transaction


All molecules stored in: untestedMolFile.txt


In [6]:
top100MolsSmiles = top100Mols['SMILES']
top100MolsSmiles.to_csv('top100UntestedMolecules.csv', index=False)

In [3]:
alldesc = ml.createDescriptorDf('top100UntestedMolecules')


Traceback (most recent call last):
  File "/Users/timstassen/Desktop/8CC00/code/groupAssignment/MachineLearning.py", line 83, in getMolDescriptors
    val = fn(mol)
          ^^^^^^^
  File "/Users/timstassen/opt/anaconda3/envs/rdkit/lib/python3.11/site-packages/rdkit/Chem/GraphDescriptors.py", line 712, in BertzCT
    return _CalculateEntropies(connectionDict, atomTypeDict, numAtoms)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/timstassen/opt/anaconda3/envs/rdkit/lib/python3.11/site-packages/rdkit/Chem/GraphDescriptors.py", line 581, in _CalculateEntropies
    connectionIE = totConnections * (entropy.InfoEntropy(numpy.array(connectionList)) +
                                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


In [4]:
allAHDL1inhibitors = ut.importFiles(nrFiles=2)
ut.writeNewMolfile(allAHDL1inhibitors)

2023-06-16 10:40:46.821 python[16775:710441] +[CATransaction synchronize] called within transaction
2023-06-16 10:40:47.047 python[16775:710441] +[CATransaction synchronize] called within transaction
2023-06-16 10:40:52.187 python[16775:710441] +[CATransaction synchronize] called within transaction
2023-06-16 10:40:52.360 python[16775:710441] +[CATransaction synchronize] called within transaction


All molecules stored in: AllTestedMols.txt


In [5]:
allDescrs = ut.createDescriptorDf()


In [6]:
x = ut.generateMorganFingerprint()


In [7]:
y = allAHDL1inhibitors[1][1:].astype(int)
sum(y)/len(y)
x, xDims = ut.CombineDescriptorsAndFigerprints(x,allDescrs)
xScale = ut.scaleData(x,scaletype='normalize')
xPrep, _ = ut.PCAfeatureReduction(xScale, 0.9)
xTrain, xTest, yTrain, yTest, cv = ut.splitTrainTestData(xPrep,y, printSplit=True)




Fold_1
TRAIN: [   0    1    2 ... 1597 1598 1599]
TEST: [   6   16   23   26   32   34   35   36   37   39   40   41   42   43
   48   53   55   59   63   67   71   84   87   94   98  110  119  123
  129  131  135  139  141  142  155  162  164  170  177  184  186  187
  192  200  204  205  210  214  218  233  236  240  242  261  263  288
  292  297  299  308  316  318  321  323  329  348  349  352  355  363
  367  373  377  385  387  390  392  393  396  397  406  408  414  423
  425  436  439  442  449  457  458  460  461  462  464  467  470  471
  476  483  484  492  503  504  507  510  522  523  528  539  540  546
  554  558  562  576  584  586  590  594  604  605  607  609  614  615
  626  628  635  642  648  652  656  663  665  680  681  683  687  692
  702  709  710  716  729  732  739  742  743  745  746  750  753  761
  766  768  774  777  780  798  813  826  839  840  842  847  848  859
  864  873  877  881  893  895  899  901  904  909  910  913  915  922
  925  927  928  930

In [8]:
model, bestParam, RFgrid, param_grid, BestScoreRFgrid = ut.trainRF(xTrain, yTrain, cv)

# ut.saveTrainedModel(model, 'RandomForestClassifier') # optional
pred, balAcc, pred_prob = ut.testTrainedModel(xTest, yTest, model=model, savedModelfilename=None, scaledData=True, scaledDatafile=None)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


Best Random Forest Classifier: 
 Parameters: {'max_features': 178, 'n_estimators': 100} 
 balanced accuracy:  0.710084347092893


OLD Part of the QSAR_PCA file

In [13]:
da, thresholdAccuracy = ut.thresholdedAccuracy(yTest, pred, pred_prob, threshold=0.8)
thresholdAccuracy

0.8981481481481481

In [ ]:
def getMolDescriptors(mol, missingVal=None):
    ''' calculate the full list of descriptors for a molecule
    
        missingVal is used if the descriptor cannot be calculated
    '''
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res

In [ ]:
filename1 = fd.askopenfilename()
AHDL1Inhibitors1 = pd.read_csv(filename1 ,header = None)
filename2 = fd.askopenfilename()
AHDL1Inhibitors2 = pd.read_csv(filename2 ,header = None)
AHDL1Inhibitors2 = AHDL1Inhibitors2.iloc[1:, :]
AHDL1Inhibitors = pd.concat([AHDL1Inhibitors1,AHDL1Inhibitors2])
AHDL1Inhibitors = AHDL1Inhibitors.reset_index(drop=True)
# print(AHDL1Inhibitors)

In [ ]:
# create new file containing merged datasets
allTestedMolecules = AHDL1Inhibitors[0] # first 3 for testing, needs to change for all molecules (remove[0:4])
MolList = allTestedMolecules.values.tolist()
with open('AllTestedMols.txt', 'w') as fp:
    for mol in MolList:
        # write each item on a new line
        fp.write("%s\n" % mol)
    print('Done')

In [ ]:


suppl = Chem.SmilesMolSupplier('AllTestedMols.txt')
mols = [m for m in suppl]
# len(mols)


In [ ]:
RDLogger.DisableLog('rdApp.*')
allDescrs = [getMolDescriptors(m) for m in mols]
allDescrsDf = pd.DataFrame(allDescrs)
allDescrsDf.head()
# print(np.dtype(allDescrsDf))

### Generate Fingerprints

In [ ]:
def rdkit_numpy_convert(fp):
    output = []
    for f in fp:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [ ]:
fingerp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols]
x = rdkit_numpy_convert(fingerp)
print(x)

In [ ]:
fpgen = AllChem.GetRDKitFPGenerator()
fps = [fpgen.GetFingerprint(x) for x in mols]
print(fps)


In [ ]:
fpgen = AllChem.GetMorganGenerator(radius=2)
fp1 = fpgen.GetSparseCountFingerprint(mols[1])
# check imbalance dataset:
y = AHDL1Inhibitors[1][1:].astype(int)
sum(y)/len(y)
# mild imbalance

Use data for Machine Learning: 
- Random forest
- SVM
- knn

In [ ]:
# split data in train and test sets. Set the testset size to 20%
seed = 13
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.20, random_state=seed)
# create folds for cross- validation
cv = StratifiedKFold(n_splits=5, shuffle=True,random_state=seed)

In [ ]:
# print out ids of folds
for i, (train_index, test_index) in enumerate(cv.split(xTrain, yTrain)):
    print("\nFold_" + str(i+1))
    print("TRAIN:", train_index)
    print("TEST:", test_index)

In [ ]:
# Scale inputs
scale = StandardScaler().fit(xTrain)
xTrain = scale.transform(xTrain)


In [ ]:
# save data for future use
joblib.dump(scale, "Fingerprints.pkl", compress=3)

In [ ]:
# create grid search dictionary
param_grid = {"max_features": [xTrain.shape[1] // 10, xTrain.shape[1] // 7, 
                               xTrain.shape[1] // 5, xTrain.shape[1] // 3], 
              "n_estimators": [100, 250, 500]}

In [ ]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [ ]:
# run model building
m.fit(xTrain, yTrain)

In [ ]:
m.best_params_

In [ ]:
m.best_score_

In [ ]:
m.cv_results_


In [ ]:
joblib.dump(m, "RFmodelMorganFingerprint.pkl", compress=3)

Load model (also to check if it works)

In [ ]:
scale = joblib.load("Fingerprints.pkl")
# scale descriptors of the test set compounds
xTest = scale.transform(xTest)
# predict logBB class
predRF = m.predict(xTest)
predRF

In [ ]:
accuracy_score(yTest, predRF)

In [ ]:
# if the model includes several ones like RF models or consensus models (or for probabilistic models)
# we can calculate consistency of predictions amongs those models and use it for estimation of applicability domain
pred_prob = m.predict_proba(xTest)
pred_prob

In [ ]:
# setup threshold
threshold = 0.8
# calc maximum predicted probability for each row (compound) and compare to the threshold
da = np.amax(pred_prob, axis=1) > threshold
da

In [ ]:
# calc statistics
# yTest 
# np.asarray(yTest)[da]
accuracy_score(np.asarray(yTest)[da], predRF[da])

In [ ]:
# calc coverage
sum(da) / len(da)

SVM approach

In [ ]:
# create grid search dictionary
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}
# setup model building
svm = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, n_jobs=2, cv=cv, verbose=1)

In [ ]:
# run model building
svm.fit(xTrain, yTrain)

In [ ]:
svm.best_params_

In [ ]:
svm.best_score_

In [ ]:
# save model
joblib.dump(svm, "SVMmorganFingerprint.pkl", compress=3)

In [ ]:
# predict logBB for the test set compounds
pred_svm = svm.predict(xTest)
pred_svm


In [ ]:
# calc statistics
print("Accuracy = ", accuracy_score(yTest, pred_svm))
print("MCC = ", matthews_corrcoef(yTest, pred_svm))
print("Kappa = ", cohen_kappa_score(yTest, pred_svm))

In [ ]:
# estimate applicability domain and calc stat
pred_prob = svm.predict_proba(xTest)
pred_prob

In [ ]:
da = np.amax(pred_prob, axis=1) > threshold

In [ ]:
print("Accuracy = ", accuracy_score(np.asarray(yTest)[da], pred_svm[da]))
print("MCC = ", matthews_corrcoef(np.asarray(yTest)[da], pred_svm[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(yTest)[da], pred_svm[da]))
print("Coverage = ", sum(da) / len(da))

Add the descriptors to the model

In [ ]:
xNew = np.concatenate((x,allDescrsDf), axis=1)
xNew.shape

Performing PCA on data

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
df = pd.DataFrame(xNew)
df_std = StandardScaler().fit_transform(df)
df_std =  pd.DataFrame(df_std)

pca = PCA()
principalComponents = pca.fit_transform(df_std)
principalDF = pd.DataFrame(data=principalComponents)

explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
i = 0
sumVar = 0
while sumVar<0.999:
    sumVar = cumulative_variance_ratio[i]
    i +=1

xNew = df_std.iloc[:,:i]


In [ ]:
xNtr, xNts, yNtr, yNts = train_test_split(xNew, y, test_size=0.20, random_state=seed)
scale = StandardScaler().fit(xNtr)
xNtr = scale.transform(xNtr)

In [ ]:
# create grid search dictionary
param_grid = {"max_features": [xNtr.shape[1] // 10, xNtr.shape[1] // 7, xNtr.shape[1] // 5, xNtr.shape[1] // 3], 
              "n_estimators": [100, 250, 500]}

In [ ]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [ ]:
# run model building
m.fit(xNtr, yNtr)

In [ ]:
m.best_score_

In [ ]:
# scale descriptors of the test set compounds
xNts = scale.transform(xNts)
# predict
pred = m.predict(xNts)
pred

In [ ]:
# calc statistics
print("Accuracy = ", accuracy_score(yNts, pred))
print("MCC = ", matthews_corrcoef(yNts, pred))
print("Kappa = ", cohen_kappa_score(yNts, pred))

In [ ]:
# estimate applicability domain and calc stat
pred_prob = m.predict_proba(xNts)
da = np.amax(pred_prob, axis=1) > threshold

print("Accuracy = ", accuracy_score(np.asarray(yNts)[da], pred[da]))
print("MCC = ", matthews_corrcoef(np.asarray(yNts)[da], pred[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(yNts)[da], pred[da]))
print("Coverage = ", sum(da) / len(da))

In [ ]:
# rebuild RF model manually using best parameters to be able to extract additional information from the model
rf = RandomForestClassifier(n_estimators=m.best_params_["n_estimators"], 
                           max_features=m.best_params_["max_features"],
                           random_state=seed)
rf.fit(xNtr, yNtr)

In [ ]:
m.best_score_

In [ ]:
imp = rf.feature_importances_
imp

In [ ]:
indices = np.argsort(imp)[::-1]

print("Feature ranking:")

# print top 10 features
for i in range(10):
    print("%d. feature %d (%f)" % (i + 1, indices[i], imp[indices[i]]))

features 1-2028 are different Morgan fingerprints

In [ ]:
data = importFiles()